In [21]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("../../")))
import pandas as pd
from src.comparisons.household import Kitchen, Bathroom

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from src.recommendation_engine.comparing_devices import comparing_devices
from src.recommendation_engine.recommend_best_device_per_group import recommend_best_per_group
from src.recommendation_engine.user_profile_model import train_profile_model
import pickle
from src.recommendation_engine.predict_profile_for_user import predict_profile_for_user
from src.recommendation_engine.validate_user_input import validate_user_input

In [22]:
df = pd.read_parquet("../../Data/GRID/grid_with_profiles.parquet")
df[df.select_dtypes(include=['float']).columns] = df.select_dtypes(include=['float']).round(1)

# delete balanced profiles
df = df[df["profile"] != "Balanced"]
df.drop(columns=["optimal_score", "unique_parameter",], inplace=True)

In [23]:
# # test cooking devices
# df_cooking = Kitchen.compare_cooking_devices(time_minutes=30)
# df_cooking_2 = df_cooking[["name", "energy_kwh", "cost_pln", "time_min", "co2_emission_kg", "device_cost", "normalized_failure_rate"]]

# # df_cooking_2.to_csv("cooking_compare.csv", index=False)
# df_cooking_2

In [24]:
base_columns = ["cost_pln", "co2_emission_kg", "normalized_comfort", "normalized_failure_rate", "device_cost"]
specific_columns = ["heating_quality", "cooking_quality", "computing_quality", "cooling_quality"]
all_features = base_columns + specific_columns

In [25]:
# print unique values from columns in all_features and order them

for column in all_features:
    unique_values = df[column].unique()
    unique_values.sort()
    print(f"{column}: {unique_values}")

cost_pln: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
co2_emission_kg: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalized_comfort: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
normalized_failure_rate: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
device_cost: [-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
heating_quality: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  nan]
cooking_quality: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  nan]
computing_quality: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  nan]
cooling_quality: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  nan]


In [26]:
# model, y_pred, report = train_profile_model(df)
# report

In [27]:
# with open('../../models/user_profile_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [28]:
# with open("../../models/feature_order_user_profile_model.pkl", "wb") as f:
#     pickle.dump(all_features, f)

In [29]:
recommend_best_per_group("Saver", df=df, comparing_dict=comparing_devices)

{'boiling_water': 'GasHob',
 'cooking': 'GasHob',
 'heating_food': 'Microwave',
 'making_coffee': 'ElectricMokaPot',
 'multicookers': 'ThermomixTM6',
 'water_heating': 'HeatPump',
 'bathing': 'Shower',
 'bathroom_heating': 'LadderHeater',
 'workstation': 'LaptopWithMonitor',
 'cooling': 'Fan'}

### Przypisywanie uytkownika na podstawie ankiety

In [30]:
with open("../../models/user_profile_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("../../models/feature_order_user_profile_model.pkl", "rb") as f:
    feature_order = pickle.load(f)

user_input = {
    "cost_pln": 1,          # 1–10
    "co2_emission_kg": 5,
    "normalized_comfort": 8,
    "normalized_failure_rate": 2,
    "device_cost": -5,      # -10 do 10
    "cooking_quality": 9
}

# Predykcja
predicted_profile = predict_profile_for_user(user_input=user_input,
                                             model=model,
                                             feature_order=feature_order,)

print("Przewidziany profil:", predicted_profile)


Przewidziany profil: QualitySeeker
